## Apply Harmony batch effect correction

Here, we apply Harmony in two distinct ways:

1. Per plate harmony using single cell normalized profiles adjusting for Well
2. All plate harmony using concatenated single cell normalized profiles adjusting for Plate and Well

We use the [`harmonypy`](https://github.com/slowkow/harmonypy) implementation (version 0.0.5) of the Harmony algorithm introduced in [Korsunsky et al. 2019](https://doi.org/10.1038/s41592-019-0619-0)

In [1]:
import pathlib
import numpy as np
import pandas as pd

import harmonypy as hm
from sklearn.decomposition import PCA

from pycytominer.cyto_utils import infer_cp_features

In [2]:
np.random.seed(123)

In [3]:
# Load constants
data_dir = pathlib.Path("../data")
feature_select_summary_file = pathlib.Path("tables/feature_select_summary.csv")
output_file_suffix = "dmso_harmony_normalized.csv"
output_file_inverse_suffix = "dmso_inverse_harmony_normalized.csv"
minimum_times_selected = 2
num_pcs = 20

harmony_random_state = 0
harmony_adjust_vars_perplate = ["Image_Metadata_Well"]
harmony_adjust_vars_full = ["Image_Metadata_Well", "Metadata_Plate"]

In [4]:
# Load data
data_dir = pathlib.Path("../data")
data_files = [x for x in data_dir.iterdir() if "_dmso_normalized.csv" in str(x)]
data_files

[PosixPath('../data/SQ00015145_dmso_normalized.csv'),
 PosixPath('../data/SQ00015143_dmso_normalized.csv'),
 PosixPath('../data/SQ00015142_dmso_normalized.csv'),
 PosixPath('../data/SQ00015201_dmso_normalized.csv'),
 PosixPath('../data/SQ00015144_dmso_normalized.csv')]

In [5]:
# Perform feature selection
feature_df = pd.read_csv(feature_select_summary_file, index_col=0)

feature_df = (
    feature_df
    .assign(times_selected=feature_df.sum(axis="columns"))
    .assign(selected=0)
)

# Load feature selection options
feature_df.loc[feature_df.times_selected > minimum_times_selected, "selected"] = 1
feature_df = feature_df.query("selected == 1")

print(feature_df.shape)
feature_df.head()

(355, 7)


,SQ00015145,SQ00015143,SQ00015201,SQ00015142,SQ00015144,times_selected,selected
Image_Metadata_Well,1,1,1,1,1,5,1
Cells_AreaShape_Eccentricity,1,1,1,1,1,5,1
Cells_AreaShape_Extent,1,1,1,1,1,5,1
Cells_AreaShape_FormFactor,1,1,1,1,1,5,1
Cells_AreaShape_Solidity,1,1,1,1,1,5,1


In [6]:
# Perform Harmony normalization per plate
all_dfs = []
for file in data_files:
    # Extract plate from file name
    plate = str(file).split("/")[-1].split("_")[0]
    print(f"Now processing {plate}...")
    
    # Load data and apply feature selection
    df = pd.read_csv(file).reindex(feature_df.index, axis="columns")
    
    # Extract feature types
    morphology_features = infer_cp_features(df, compartments=["Cells", "Cytoplasm", "Nuclei"])
    metadata_cols = ["Image_Metadata_Well"] + infer_cp_features(df, metadata=True)
    
    # Fit PCA
    pca = PCA(n_components=num_pcs)
    pca.fit(df.loc[:, morphology_features])
    
    # Transform PCA
    pc_df = pca.transform(df.loc[:, morphology_features])
    pc_df = pd.DataFrame(pc_df).add_prefix("pca_") 
    
    # Apply harmony per plate
    harmony_out = (
        hm.run_harmony(
            data_mat=pc_df,
            meta_data=df.loc[:, metadata_cols],
            vars_use=harmony_adjust_vars_perplate,
            random_state=harmony_random_state
        )
    )
    
    # Compile harmony output file
    harmony_df = pd.concat(
        [
            df.loc[:, metadata_cols],
            pd.DataFrame(harmony_out.Z_corr).transpose().add_prefix("harmonized_")
        ],
        axis="columns"
    )

    # Output harmonized file
    output_file = pathlib.Path(f"{data_dir}/{plate}_{output_file_suffix}")
    harmony_df.to_csv(output_file, index=False, sep=",")

    # Apply an inverse transform to get back to original feature space
    inverse_harmony_df = pd.concat(
        [
            df.loc[:, metadata_cols],
            pd.DataFrame(pca.inverse_transform(harmony_out.Z_corr.transpose()), columns=morphology_features)
        ],
        axis="columns"
    )
    
    # Output inverse harmonized file
    output_file = pathlib.Path(f"{data_dir}/{plate}_{output_file_inverse_suffix}")
    inverse_harmony_df.to_csv(output_file, index=False, sep=",")

    # Append all cells to all dfs
    all_dfs.append(df)
    del df
    del harmony_df
    del inverse_harmony_df

Now processing SQ00015145...


2020-10-30 17:27:15,960 - harmonypy - INFO - Iteration 1 of 10
2020-10-30 17:27:56,921 - harmonypy - INFO - Iteration 2 of 10
2020-10-30 17:28:37,848 - harmonypy - INFO - Iteration 3 of 10
2020-10-30 17:29:03,712 - harmonypy - INFO - Iteration 4 of 10
2020-10-30 17:29:29,217 - harmonypy - INFO - Iteration 5 of 10
2020-10-30 17:29:47,554 - harmonypy - INFO - Iteration 6 of 10
2020-10-30 17:30:06,083 - harmonypy - INFO - Iteration 7 of 10
2020-10-30 17:30:20,770 - harmonypy - INFO - Iteration 8 of 10
2020-10-30 17:30:35,298 - harmonypy - INFO - Converged after 8 iterations


Now processing SQ00015143...


2020-10-30 17:32:21,253 - harmonypy - INFO - Iteration 1 of 10
2020-10-30 17:32:59,124 - harmonypy - INFO - Iteration 2 of 10
2020-10-30 17:33:36,226 - harmonypy - INFO - Iteration 3 of 10
2020-10-30 17:34:13,731 - harmonypy - INFO - Iteration 4 of 10
2020-10-30 17:34:41,853 - harmonypy - INFO - Iteration 5 of 10
2020-10-30 17:34:59,307 - harmonypy - INFO - Iteration 6 of 10
2020-10-30 17:35:12,901 - harmonypy - INFO - Iteration 7 of 10
2020-10-30 17:35:25,203 - harmonypy - INFO - Converged after 7 iterations


Now processing SQ00015142...


2020-10-30 17:37:10,940 - harmonypy - INFO - Iteration 1 of 10
2020-10-30 17:37:52,702 - harmonypy - INFO - Iteration 2 of 10
2020-10-30 17:38:34,308 - harmonypy - INFO - Iteration 3 of 10
2020-10-30 17:39:15,669 - harmonypy - INFO - Iteration 4 of 10
2020-10-30 17:39:44,060 - harmonypy - INFO - Iteration 5 of 10
2020-10-30 17:40:10,095 - harmonypy - INFO - Iteration 6 of 10
2020-10-30 17:40:34,322 - harmonypy - INFO - Iteration 7 of 10
2020-10-30 17:40:49,458 - harmonypy - INFO - Iteration 8 of 10
2020-10-30 17:41:02,823 - harmonypy - INFO - Converged after 8 iterations


Now processing SQ00015201...


2020-10-30 17:42:38,563 - harmonypy - INFO - Iteration 1 of 10
2020-10-30 17:43:11,218 - harmonypy - INFO - Iteration 2 of 10
2020-10-30 17:43:44,294 - harmonypy - INFO - Iteration 3 of 10
2020-10-30 17:44:10,458 - harmonypy - INFO - Iteration 4 of 10
2020-10-30 17:44:31,630 - harmonypy - INFO - Iteration 5 of 10
2020-10-30 17:44:46,454 - harmonypy - INFO - Iteration 6 of 10
2020-10-30 17:45:01,338 - harmonypy - INFO - Iteration 7 of 10
2020-10-30 17:45:13,576 - harmonypy - INFO - Iteration 8 of 10
2020-10-30 17:45:24,066 - harmonypy - INFO - Converged after 8 iterations


Now processing SQ00015144...


2020-10-30 17:46:57,346 - harmonypy - INFO - Iteration 1 of 10
2020-10-30 17:47:35,417 - harmonypy - INFO - Iteration 2 of 10
2020-10-30 17:48:13,929 - harmonypy - INFO - Iteration 3 of 10
2020-10-30 17:48:48,846 - harmonypy - INFO - Iteration 4 of 10
2020-10-30 17:49:11,037 - harmonypy - INFO - Iteration 5 of 10
2020-10-30 17:49:32,067 - harmonypy - INFO - Iteration 6 of 10
2020-10-30 17:49:49,551 - harmonypy - INFO - Iteration 7 of 10
2020-10-30 17:50:01,297 - harmonypy - INFO - Converged after 7 iterations


In [7]:
# Apply Harmony normalization for all plates at once
all_dfs = pd.concat(all_dfs, axis="rows").reset_index(drop=True)

# Fit PCA
pca = PCA(n_components=num_pcs)
pca.fit(all_dfs.loc[:, morphology_features])

# Transform PCA
pc_df = pca.transform(all_dfs.loc[:, morphology_features])
pc_df = pd.DataFrame(pc_df).add_prefix("pca_") 

# Apply harmony per plate
harmony_out = (
    hm.run_harmony(
        data_mat=pc_df,
        meta_data=all_dfs.loc[:, metadata_cols],
        vars_use=harmony_adjust_vars_full,
        random_state=harmony_random_state
    )
)

# Compile harmony output file
harmony_df = pd.concat(
    [
        all_dfs.loc[:, metadata_cols],
        pd.DataFrame(harmony_out.Z_corr).transpose().add_prefix("harmonized_all_plates_")
    ],
    axis="columns"
)

# Output harmonized file
output_file = pathlib.Path(f"{data_dir}/dmso_harmonized_all_plates.csv.gz")
harmony_df.to_csv(output_file, index=False, sep=",", compression="gzip")

print(harmony_df.shape)
harmony_df.head(2)

2020-10-30 17:53:36,381 - harmonypy - INFO - Iteration 1 of 10
2020-10-30 17:56:50,794 - harmonypy - INFO - Iteration 2 of 10
2020-10-30 18:00:13,211 - harmonypy - INFO - Iteration 3 of 10
2020-10-30 18:03:16,910 - harmonypy - INFO - Iteration 4 of 10
2020-10-30 18:04:31,236 - harmonypy - INFO - Iteration 5 of 10
2020-10-30 18:05:36,520 - harmonypy - INFO - Iteration 6 of 10
2020-10-30 18:06:42,069 - harmonypy - INFO - Converged after 6 iterations


(319967, 27)


,Image_Metadata_Well,Metadata_plate_map_name,Metadata_broad_sample,Metadata_mg_per_ml,Metadata_mmoles_per_liter,Metadata_solvent,Metadata_Plate,harmonized_all_plates_0,harmonized_all_plates_1,harmonized_all_plates_2,...,harmonized_all_plates_10,harmonized_all_plates_11,harmonized_all_plates_12,harmonized_all_plates_13,harmonized_all_plates_14,harmonized_all_plates_15,harmonized_all_plates_16,harmonized_all_plates_17,harmonized_all_plates_18,harmonized_all_plates_19
0,A01,C-7161-01-LM6-017,DMSO,NaN,NaN,DMSO,SQ00015145,-3.834715,-0.337823,0.409828,...,0.392792,-2.587241,-1.307714,-0.109904,1.722504,-1.566572,-1.490985,-0.441006,-1.326397,2.652354
1,A01,C-7161-01-LM6-017,DMSO,NaN,NaN,DMSO,SQ00015145,-1.635186,-2.804493,-2.582731,...,0.435962,-1.913950,-3.684907,2.152964,1.482171,-2.496940,-3.442950,3.326981,1.013545,1.841008


In [8]:
# Apply an inverse transform to get back to original feature space
inverse_harmony_df = pd.concat(
    [
        all_dfs.loc[:, metadata_cols],
        pd.DataFrame(pca.inverse_transform(harmony_out.Z_corr.transpose()), columns=morphology_features)
    ],
    axis="columns"
)

# Output inverse harmonized file
output_file = pathlib.Path(f"{data_dir}/dmso_inverse_harmonized_all_plates.csv.gz")
inverse_harmony_df.to_csv(output_file, index=False, sep=",")

print(inverse_harmony_df.shape)
inverse_harmony_df.head(2)

(319967, 355)


,Image_Metadata_Well,Metadata_plate_map_name,Metadata_broad_sample,Metadata_mg_per_ml,Metadata_mmoles_per_liter,Metadata_solvent,Metadata_Plate,Cells_AreaShape_Eccentricity,Cells_AreaShape_Extent,Cells_AreaShape_FormFactor,...,Nuclei_Texture_SumEntropy_DNA_5_0,Nuclei_Texture_SumEntropy_Mito_20_0,Nuclei_Texture_SumEntropy_RNA_10_0,Nuclei_Texture_SumVariance_ER_20_0,Nuclei_Texture_SumVariance_Mito_20_0,Nuclei_Texture_SumVariance_RNA_20_0,Nuclei_Texture_Variance_AGP_20_0,Nuclei_Texture_Variance_DNA_20_0,Nuclei_Texture_Variance_RNA_10_0,Nuclei_Texture_Variance_RNA_20_0
0,A01,C-7161-01-LM6-017,DMSO,NaN,NaN,DMSO,SQ00015145,0.643318,-0.465800,-0.516848,...,0.103755,0.122439,0.021195,0.102766,-0.019217,0.135866,0.043713,0.147157,-0.138302,0.145783
1,A01,C-7161-01-LM6-017,DMSO,NaN,NaN,DMSO,SQ00015145,1.275632,-0.155096,-0.660541,...,0.587360,0.019732,0.350383,0.192835,0.198684,0.279934,-0.724892,0.011361,-0.309659,-0.044058
